# Интересное про python

## Скорость Numpy

NDArray работает в разы быстрее благодаря способу хранения информации

In [ ]:
import pandas as pd

In [ ]:
a = [1, 2, 3, 300]

for i in range(len(a)-1):

    print(i, id(a[i]))
    print(i+1, id(a[i+1]))
    print('delta', id(a[i+1]) - id(a[i]))
    print('--')

Python использует хитрые оптимизации и кэширует целые числа от -5 до 256 в **непрерывной** области памяти

In [ ]:
id(256) - id(0), id(257) - id(0)

При повторном вызове строчки выше, мы получим другое значение во втором элементе ответа, так как python заново создаст объект 257 где-то в куче и расстояние между ним и объектом 0 уже будет другое

In [ ]:
id(256) - id(0), id(257) - id(0)

## display()

In [ ]:
a = [1, 2, 3]
b = a
del a
print(b)
# print(a)

In [ ]:
a = [1, 2, 3]
b = a
b[0] = 10
print(a)

In [ ]:
a = [1, 2, 3]
b = a[:]
b[0] = 10
print(a)

In [ ]:
a = [2, 4, 6]
a

In [ ]:
a

In [ ]:
# Обновляем куда указывает имя "a", по хорошему старые данные должны затереться
a = [3, 5, 7]

In [ ]:
# Так как в IPython есть специальный массив куда сказываются аутпуты ячеек,
# то старое содержимое "a" никуда не делось и всё равно весит в памяти
# Out[144]

In [ ]:
df = pd.DataFrame()

In [ ]:
df

In [ ]:
# df.drop(colums=['a'], inplace=True)

In [ ]:
display([1, 2, 3])

# Урок 3. Классификация

In [ ]:
import numpy as np
import pandas as pd

## Смотрим на данные

In [ ]:
# Качаем данные
# !gdown 1nCHCT5XWio5fSN0mYNwRbEzTGL_sIcN4

Датасет заявок на кредит с информацией о клиента и флагом одобрение или отказ по заявке

In [ ]:
#Считываем скачанный csv файл
df = pd.read_csv('loan_data.csv')

In [ ]:
# Посмотрим на него
display(df)

In [ ]:
df.head(10)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Заведём константу, чтобы каждый раз не вспоминать как называется колонка с таргетом
TARGET = 'loan_status'

In [ ]:
# Посмотрим на пропуски
df.isna().sum()

In [ ]:
df['age_bin'] = df['person_age'] #//6

In [ ]:
# Посмотрим на распределение среднего дохода в зависимости от возраста
df[df['person_age'] < 80].groupby('age_bin')['person_income'].mean().plot()

In [ ]:
df['age_bin'] = df['person_age'] // 6
df[df['person_age'] < 80].groupby('age_bin')['person_income'].mean().plot()

Пропусков нет - скучно :(

In [ ]:
from numpy import random
# зашумим часть данных для дальнейшего интереса :-)

# Зашумим случаный 1000 возрастов (может быть меньше с учетом коллизий индексов)
# Это даст нам случайное распределение пустот
df.loc[random.choice(range(df.shape[0]), 1000), 'person_age'] = np.nan

# Уберём значительную часть доходов у людей, которые хотят взять кредит на образование
# Это зашумление нам даст неслучайное смещение
df.loc[df[df['loan_intent'] == 'EDUCATION'].sample(3000).index, 'person_income'] = np.nan

In [ ]:
df.isna().sum()


In [ ]:
# Проверим гипотезы про случайность/неслучайность получившихся пустот

print('Target rate all:', df[TARGET].mean())

print('--'*20)

print('Target rate income null:', df[df['person_income'].isna()][TARGET].mean())
print('Target rate income not null:', df[df['person_income'].notna()][TARGET].mean())

print('--'*20)

print('Target rate age null:', df[df['person_age'].isna()][TARGET].mean())
print('Target rate age not null:', df[df['person_age'].notna()][TARGET].mean())

## Попробуем обучить LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
# Разобъем на тренировочную и тестовую выборки
train, test = train_test_split(df, test_size=0.25, random_state=43)

In [ ]:
# Для простоты возьмём несколько числовых колонок в качестве факторов
features = ['loan_amnt', 'loan_int_rate'] # loan_amnt - сумма кредита, loan_int_rate - процентная ставка

In [ ]:
X_train = train[features]
y_train = train[TARGET] #loan status - статус кредита (наш таргет)

X_test = test[features]
y_test = test[TARGET]

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
def least_squared_estimator(X, y, learn_intercept=False):
  X = np.array(X)
  if learn_intercept:
    X = np.concatenate([np.ones((X.shape[0], 1), ), X ], axis=1)
  coefs = np.linalg.inv(X.T @ X) @ X.T @ y
  return coefs

In [ ]:
coefs = least_squared_estimator(X_train, y_train, learn_intercept=True)
f = lambda x: coefs[0] + coefs[1] * x[:, 0] + coefs[2] * x[:, 1]

print(f)

In [ ]:
coefs

In [ ]:
train['predict'] = lr.predict(X_train)
test['predict'] = lr.predict(X_test)


train['predict_proba'] = lr.predict_proba(X_train)[:, 1]
test['predict_proba'] = lr.predict_proba(X_test)[:, 1]

In [ ]:
train['predict_naive'] = f(X_train.values)
test['predict_naive'] = f(X_test.values)

In [ ]:
import matplotlib as mpl

train_sample = train.sample(2500)
test_sample = test.sample(2500)

print(train_sample)

Посмотрим как мы попали на Train

In [ ]:
cmap = sns.color_palette('ch:', as_cmap= True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize())
scatter = sns.scatterplot(data=train_sample, x='loan_amnt', y='loan_int_rate', hue=TARGET, hue_norm=sm.norm, palette= cmap, legend= False)
cbar = plt.colorbar(sm, ax = plt.gca())
cbar.set_label('hue')

In [ ]:
#плохой способ, predict 0,5
cmap= sns.color_palette('ch:', as_cmap= True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize())
scatter = sns.scatterplot(data=train_sample, x='loan_amnt', y='loan_int_rate', hue='predict', hue_norm=sm.norm, palette= cmap, legend= False)
cbar = plt.colorbar(sm, ax = plt.gca())
cbar.set_label('hue')

In [ ]:
#хороший способ, порог настраиваем сами
train_sample['new_predict'] = (train_sample['predict_proba'] > 0.44).astype(int)
cmap= sns.color_palette('ch:', as_cmap= True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize())
scatter = sns.scatterplot(data=train_sample, x='loan_amnt', y='loan_int_rate', hue='new_predict', hue_norm=sm.norm, palette= cmap, legend= False)
cbar = plt.colorbar(sm, ax = plt.gca())
cbar.set_label('hue')

In [ ]:
#градиент цвета, норм подход
cmap= sns.color_palette('ch:', as_cmap= True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize())
scatter = sns.scatterplot(data=train_sample, x='loan_amnt', y='loan_int_rate', hue='predict_proba', hue_norm=sm.norm, palette= cmap, legend= False);
cbar = plt.colorbar(sm, ax = plt.gca())
cbar.set_label('hue')

In [ ]:
#наивный подход 
train_sample['new_predict'] = (train_sample['predict_naive'] > 0.1).astype(int)
cmap= sns.color_palette('ch:', as_cmap= True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize())
scatter = sns.scatterplot(data=train_sample, x='loan_amnt', y='loan_int_rate', hue='new_predict', hue_norm=sm.norm, palette= cmap, legend= False);
cbar = plt.colorbar(sm, ax = plt.gca())
cbar.set_label('hue')

Посмотрим как мы попали на Test

In [ ]:
cmap = sns.color_palette('ch:', as_cmap= True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize())
scatter = sns.scatterplot(data=test_sample, x='loan_amnt', y='loan_int_rate', hue=TARGET, hue_norm=sm.norm, palette= cmap, legend= False);
cbar = plt.colorbar(sm, ax = plt.gca())
cbar.set_label('hue')

In [ ]:
cmap= sns.color_palette('ch:', as_cmap= True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize())
scatter = sns.scatterplot(data=test_sample, x='loan_amnt', y='loan_int_rate', hue='predict', hue_norm=sm.norm, palette= cmap, legend= False);
cbar = plt.colorbar(sm, ax = plt.gca())
cbar.set_label('hue')

In [ ]:
cmap= sns.color_palette('ch:', as_cmap= True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize())
scatter = sns.scatterplot(data=test_sample, x='loan_amnt', y='loan_int_rate', hue='predict_proba', hue_norm=sm.norm, palette= cmap, legend= False);
cbar = plt.colorbar(sm, ax = plt.gca())
cbar.set_label('hue')

### Посчитаем Метрики

In [ ]:
# Для начала заимпортим их
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Нарисуем Confusion Matrix
ConfusionMatrixDisplay(confusion_matrix(train[TARGET], train['predict'])).plot()

Верхняя левая (25494):
Истинный класс: 0
Предсказанный класс: 0
Значение: Это количество объектов, которые на самом деле относятся к классу 0, и модель правильно предсказала их как класс 0. Это Истинно Отрицательные (True Negatives, TN). Модель молодец.

Верхняя правая (717):
Истинный класс: 0
Предсказанный класс: 1
Значение: Это количество объектов, которые на самом деле относятся к классу 0, но модель ошибочно предсказала их как класс 1. Это Ложно Положительные (False Positives, FP), или "ошибка I рода". Модель ошиблась.

Нижняя левая (6300):
Истинный класс: 1
Предсказанный класс: 0
Значение: Это количество объектов, которые на самом деле относятся к классу 1, но модель ошибочно предсказала их как класс 0. Это Ложно Отрицательные (False Negatives, FN), или "ошибка II рода". Модель ошиблась (возможно, это самая критичная ошибка, если класс 1 - это что-то важное, например, болезнь).

Нижняя правая (1239):
Истинный класс: 1
Предсказанный класс: 1
Значение: Это количество объектов, которые на самом деле относятся к классу 1, и модель правильно предсказала их как класс 1. Это Истинно Положительные (True Positives, TP). Модель молодец.

In [ ]:
# train
accuracy_score(train[TARGET], train['predict'])

In [ ]:
# train
accuracy_score(train[TARGET], [0] * train.shape[0])

In [ ]:
# test
accuracy_score(test[TARGET], test['predict'])

In [ ]:
accuracy_score(train[TARGET], (train['predict_proba'] > 0.44).astype(int))

In [ ]:
# train
precision_score(train[TARGET], train['predict'])

In [ ]:
# test
precision_score(test[TARGET], test['predict'])

In [ ]:
# train
recall_score(train[TARGET], train['predict'])

In [ ]:
# test
recall_score(test[TARGET], test['predict'])

In [ ]:
# train
roc_auc_score(train[TARGET], train['predict'])

In [ ]:
# test
roc_auc_score(test[TARGET], test['predict'])

In [ ]:
# train
roc_auc_score(train[TARGET], train['predict_proba'])

In [ ]:
# test
roc_auc_score(test[TARGET], test['predict_proba'])

In [ ]:
roc_auc_score(train[TARGET], train['predict_naive'])

In [ ]:
roc_auc_score(test[TARGET], test['predict_naive'])

In [ ]:
from tqdm.auto import tqdm
stats = []
for threshold in tqdm(np.linspace(0.01, 1, 100)):
    stats.append({
        'threshold': threshold,
        'accuracy': accuracy_score(train[TARGET], (train['predict_proba'] > threshold).astype(int)),
        'accuracy_naive': accuracy_score(train[TARGET], (train['predict_naive'] > threshold).astype(int)),
        'precision': precision_score(train[TARGET], (train['predict_proba'] > threshold).astype(int), zero_division=0),
        'precision_naive': precision_score(train[TARGET], (train['predict_naive'] > threshold).astype(int), zero_division=0),
        'recall': recall_score(train[TARGET], (train['predict_proba'] > threshold).astype(int), zero_division=0),
        'recall_naive': recall_score(train[TARGET], (train['predict_naive'] > threshold).astype(int), zero_division=0),
    })
stats = pd.DataFrame(stats)

In [ ]:
sns.lineplot(data=stats, x='threshold', y='accuracy', label='accuracy')
sns.lineplot(data=stats, x='threshold', y='accuracy_naive',  label='accuracy_naive')

In [ ]:
sns.lineplot(data=stats, x='threshold', y='precision', label='precision')
sns.lineplot(data=stats, x='threshold', y='precision_naive',  label='precision_naive')

In [ ]:
sns.lineplot(data=stats, x='threshold', y='recall', label='recall')
sns.lineplot(data=stats, x='threshold', y='recall_naive',  label='recall_naive')

### Таргет рейт в бинах скоров модели (калибровка)

In [ ]:
# Посчитаем равномерные бины по скору на train
_, bins = pd.qcut(train['predict_proba'], q=20, retbins=True)

# Пременим их к train и test
train['proba_bin'] = pd.cut(train['predict_proba'], bins=bins)
test['proba_bin'] = pd.cut(test['predict_proba'], bins=bins)

In [ ]:
# Внутри бинов посчитаем средний таргет и среднюю predict_proba
train_stats = train.groupby(['proba_bin'])[['predict_proba', TARGET]].mean()
display(train_stats)

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression

base_model = LogisticRegression()
base_model.fit(train[['predict_proba']], train[TARGET])

calibration = CalibratedClassifierCV(estimator=base_model, cv='prefit')
calibration.fit(train[['predict_proba']], train[TARGET])

train['predict_proba_calib'] = calibration.predict_proba(train[['predict_proba']])[:, 1]
test['predict_proba_calib'] = calibration.predict_proba(test[['predict_proba']])[:, 1]

In [ ]:
train_stats.plot()
plt.xticks(rotation=45)

In [ ]:
valid_stats = test.groupby(['proba_bin'])[['predict_proba', TARGET]].mean()
valid_stats.plot()
plt.xticks(rotation=45)

# Домашнее Задание

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd

## Logistic Regression (Своими Силами)

1. Дописать код градиентного спуска ниже
2. Поиграться с датасетом, предложеннным выше
3. Построить свою версию Логистической регресии (на каком-то одном числовом факторе без пропусков), сравнить результаты с LogisticRegression из SKlearn (Оценить метрики - ROC AUC, подобрать порог и Построить Confusion Matrix, а так же производные метрики от неё )
4. Заполнить пропуски, обработать категориальные переменные, обучить свою версию модели и Logistic Regression, сравнить качество между ними (см п.3) + с моделями из п.3 (как добавление бОльшего количества факторов влияет на качество модели)
5. *Поиграться с нормировкой данных (sklearn.model_selection.StandardScaler) оценить эффект на веса модели и её качество
6. Разбить факторы на Клиентские/Продуктовые/Кредитная История и построить по одной модели на каждом факторе. Затем построить мета-модель (интегральная модель) на трёх факторах - скорах метамоделях (дрейверы) - Сравнить результаты по каждому из драйеров + интегральной модели с одной моделью на всём факторном прстранстве.
7. *Привести график качества (ROC AUC) модели на фиксированной тренировочной выборке в зависимости от количества наблюдений, которые используются при обучении (train выборка). Например: 100, 500, 1000 и тд.

*- задания со звёздочкой, не обязательно, но довольно интересные

In [ ]:
def sigmoid(z):
    return (1+np.exp(-z))**-1

def logloss(w, X, y):
    return np.log2(1 + np.exp(-(X @ w)*y)).mean()

In [ ]:
# Реализуем собственный класс логистической регрессии
class MyLogisticRegression:
    def __init__(self, lr=1e-4, bias=True):
      self.bias = bool(bias)
      self.weights = None
      self.lr = lr

    def preproc_x(self, X):
      # Добавляем bias если требуется
      return np.hstack([np.ones((X.shape[0], 1)), X]) if self.bias else X

    def fit(self, X, y, c_epoch=1000):
      # Инициализируем веса
      self.weights = np.ones(X.shape[1]+self.bias)

      self.X = self.preproc_x(X)
      self.y = y


      # Стохастический градиентный спуск
      # Итерируемся по количеству эпох
      for i in range(c_epoch):
        # Перешмешиваем наблюдения случайным образом
        ids = list(range(self.X.shape[0]))
        np.random.shuffle(ids)

        # итерируемся по наблюдениям
        for j in ids:
          p = sigmoid(np.dot(self.X[j], self.weights))
          # !TODO: Реализовать самостоятельно
          self.weights += self.lr * (self.y[j] - p) * self.X[j]

        print(i , ':', logloss(self.weights, self.X, self.y))

    def predict_proba(self, X):
      X = self.preproc_x(X)
      return sigmoid((X @ self.weights)*1)

    def predict(self, X, threshold = 0.5): 
      # X = self.preproc_x(X)
      # preds = (sigmoid((X @ self.weights)*1) > 0.5).astype(int)
      # preds[preds == 0] = -1
      probs = self.predict_proba(X)
      
      return (probs > threshold).astype(int)

In [ ]:
# Считываем скачанный csv файл
df = pd.read_csv('loan_data.csv')

In [ ]:
from numpy import random
# зашумим часть данных для дальнейшего интереса :-)

# Зашумим случаный 1000 возрастов (может быть меньше с учетом коллизий индексов)
# Это даст нам случайное распределение пустот
df.loc[random.choice(range(df.shape[0]), 1000), 'person_age'] = np.nan

# Уберём значительную часть доходов у людей, которые хотят взять кредит на образование
# Это зашумление нам даст неслучайное смещение
df.loc[df[df['loan_intent'] == 'EDUCATION'].sample(3000).index, 'person_income'] = np.nan

In [ ]:
# Разбить на train и test выборки
from sklearn.impute import SimpleImputer

train_imputed = train.copy()
test_imputed = test.copy()

### Независимые переменные (Features) + const strings

In [ ]:
# features = ['loan_amnt', 'loan_int_rate']
feature_1 = 'loan_percent_income'
feature_2 = 'credit_score'
features = [feature_1, feature_2]

# CONST STRING
MY_PREDICT = 'my_predict'
MY_PREDICT_PROBA = 'my_predict_proba'

MY_NRML_PREDICT = 'my_nrml_predict'
MY_NRML_PREDICT_PROBA = 'my_nrml_predict_proba'

MY_SCALED_PREDICT = 'my_scaled_predict'
MY_SCALED_PREDICT_PROBA= 'my_scaled_proba'

SK_SCALED_PREDICT = 'sk_scaled_predict'
SK_SCALED_PERDICT_PROBA = 'sk_scaled_proba'

SKLEARN_PREDICT = 'sk_predict'
SKLEARN_PREDICT_PROBA = 'sk_predict_proba'

In [ ]:
X_train = train_imputed[features].values
y_train = train_imputed[TARGET].values
X_test = test_imputed[features].values
y_test = test_imputed[TARGET].values

## Параметры модели MyLogisticRegression (с зашумленными данными)

In [ ]:
# CONST 
LR = 0.001 
C_EPOCH = 10
THRESHOLD= 0.65

# 1-ая версия
my_lr = MyLogisticRegression(LR)
my_lr.fit(X_train, y_train, C_EPOCH)

Влияние параметров `lr` и `c_epoch`

- **`lr` (learning rate)**: Это скорость обучения модели. Чем больше значение `lr`, тем быстрее модель обновляет свои веса. Однако слишком большое значение может привести к нестабильности обучения (модель может не сойтись), а слишком маленькое — к очень медленному обучению.

- **`c_epoch` (number of epochs)**: Это количество эпох, за которые модель проходит через весь тренировочный набор данных. Большее количество эпох позволяет модели лучше подстроиться под данные, но может привести к переобучению, если модель слишком долго обучается.

In [ ]:
sk_lr = LogisticRegression(max_iter=1000)
sk_lr.fit(X_train, y_train)

In [ ]:
train_imputed[MY_PREDICT] = my_lr.predict(X_train, threshold=THRESHOLD)
train_imputed[MY_PREDICT_PROBA] = my_lr.predict_proba(X_train)
test_imputed[MY_PREDICT] = my_lr.predict(X_test, threshold=THRESHOLD)
test_imputed[MY_PREDICT_PROBA] = my_lr.predict_proba(X_test)

train_imputed[SKLEARN_PREDICT] = sk_lr.predict(X_train)
train_imputed[SKLEARN_PREDICT_PROBA] = sk_lr.predict_proba(X_train)[:, 1]
test_imputed[SKLEARN_PREDICT] = sk_lr.predict(X_test)
test_imputed[SKLEARN_PREDICT_PROBA] = sk_lr.predict_proba(X_test)[:, 1]

**my_predict:** Это бинарные предсказания модели (0 или 1), которые используются для классификации объектов.
Например, если вероятность принадлежности к классу 1 (my_predict_proba) больше 0.5, то объект классифицируется как 1, иначе как 0.
Используется для оценки метрик, таких как accuracy, precision, recall, confusion matrix.


**my_predict_proba:**
Это вероятности принадлежности объекта к положительному классу (обычно класс 1).
Используются для более тонкого анализа, например:
Построение ROC-кривой и вычисление ROC AUC.
Настройка порога классификации (например, вместо 0.5 можно выбрать 0.6 для повышения precision).
Калибровка вероятностей (например, с помощью Platt Scaling или Isotonic Regression).


Logistic Regression — это метод машинного обучения, который используется для классификации.

1. **Цель**: Определить вероятность принадлежности объекта к определённому классу (например, 0 или 1).

2. **Как работает**:
   - Модель берёт входные данные (факторы) и умножает их на веса.
   - Затем добавляется смещение (bias).
   - Результат пропускается через функцию **сигмоида**, которая преобразует его в вероятность (значение от 0 до 1).

3. **Решение**:
   - Если вероятность больше 0.5, модель предсказывает класс 1.
   - Если меньше, то класс 0.

4. **Пример**:
   - У вас есть данные о доходе и возрасте человека.
   - Модель предсказывает, одобрить ли кредит (1) или отказать (0).

### Сравнение метрик : MyLogReg vs Sk_LogReg

In [ ]:
print("MyLogisticRegression weights:", my_lr.weights)
print("Sklearn LogisticRegression weights:", np.append(sk_lr.intercept_, sk_lr.coef_[0]))

In [ ]:
metrics = {}
models = ['my', 'sk']

for model in models:
    metrics[model] = {
        'accuracy': accuracy_score(test_imputed[TARGET], test_imputed[f'{model}_predict']),
        'precision': precision_score(test_imputed[TARGET], test_imputed[f'{model}_predict']),
        'recall': recall_score(test_imputed[TARGET], test_imputed[f'{model}_predict']),
        'roc_auc': roc_auc_score(test_imputed[TARGET], test_imputed[f'{model}_predict_proba'])
    }

metrics_df = pd.DataFrame(metrics).T
print("\nMetrics comparison:")
print(metrics_df)

### Сравнение матриц ошибок для моделей

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
ConfusionMatrixDisplay(confusion_matrix(test_imputed[TARGET], test_imputed[MY_PREDICT])).plot(ax=axes[0])
axes[0].set_title('MyLogisticRegression')
ConfusionMatrixDisplay(confusion_matrix(test_imputed[TARGET], test_imputed[SKLEARN_PREDICT])).plot(ax=axes[1])
axes[1].set_title('Sklearn LogisticRegression')
plt.tight_layout()
plt.show()

### Распределения вероятностей

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(test_imputed[MY_PREDICT_PROBA], bins=50, alpha=0.5, label='My LogReg')
plt.hist(test_imputed[SKLEARN_PREDICT_PROBA], bins=50, alpha=0.5, label='Sklearn LogReg')
plt.title('Probability Distributions')
plt.xlabel('Predicted Probability')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Сравнение моделей

In [ ]:
# иногда визуальные деффекты 
test_sample = test_imputed.sample(2500)
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
cmap = sns.color_palette('ch:', as_cmap=True)
import matplotlib.colors as mcolors
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mcolors.Normalize())
scatter = sns.scatterplot(data=test_sample, x=feature_1, y=feature_2, hue=TARGET, 
                         hue_norm=sm.norm, palette=cmap)
plt.colorbar(sm, ax=plt.gca()).set_label('Target')
plt.title('Actual Target Values')

plt.subplot(2, 2, 2)
scatter = sns.scatterplot(data=test_sample, x=feature_1, y=feature_2, hue=MY_PREDICT_PROBA, 
                         hue_norm=sm.norm, palette=cmap)
plt.colorbar(sm, ax=plt.gca()).set_label('My LogReg Probability')
plt.title('MyLogisticRegression Probabilities')

plt.subplot(2, 2, 3)
scatter = sns.scatterplot(data=test_sample, x=feature_1, y=feature_2, hue=SKLEARN_PREDICT_PROBA, 
                         hue_norm=sm.norm, palette=cmap)
plt.colorbar(sm, ax=plt.gca()).set_label('Sklearn LogReg Probability')
plt.title('Sklearn LogisticRegression Probabilities')

plt.subplot(2, 2, 4)
test_sample['my_predict_threshold'] = (test_sample[MY_PREDICT_PROBA] > 0.5).astype(int)
test_sample['comparison'] = np.where(
    (test_sample['my_predict_threshold'] == test_sample[SKLEARN_PREDICT]) & 
    (test_sample['my_predict_threshold'] == test_sample[TARGET]), 
    'Both Correct', 
    np.where(
        (test_sample['my_predict_threshold'] != test_sample[SKLEARN_PREDICT]) & 
        (test_sample['my_predict_threshold'] == test_sample[TARGET]), 
        'MyLogRegr Correct', 
        np.where(
            (test_sample['my_predict_threshold'] != test_sample[SKLEARN_PREDICT]) & 
            (test_sample[SKLEARN_PREDICT] == test_sample[TARGET]), 
            'Sklearn Correct', 
            'Both Wrong'
        )
    )
)
scatter = sns.scatterplot(data=test_sample, x=feature_1, y=feature_2, hue='comparison', 
                         palette={'Both Correct': 'green', 'MyLogRegr Correct': 'blue', 
                                  'Sklearn Correct': 'orange', 'Both Wrong': 'red'})
plt.title('Comparison of Predictions')
plt.legend(title='Prediction Comparison')

plt.tight_layout()
plt.show()

## MyLogisticRegression после стандартизации

In [ ]:
from sklearn.discriminant_analysis import StandardScaler

sclr = StandardScaler()
X_train_scaled = sclr.fit_transform(X_train)
X_test_scaled = sclr.transform(X_test)

#2-ая версия
my_lr_normlalized = MyLogisticRegression(LR)
my_lr_normlalized.fit(X_train_scaled, y_train, C_EPOCH)

# my_lr.fit(X_train_scaled, y_train, C_EPOCH)

In [ ]:
train_imputed[MY_NRML_PREDICT] = my_lr_normlalized.predict(X_train_scaled, threshold = THRESHOLD)
train_imputed[MY_NRML_PREDICT_PROBA] = my_lr_normlalized.predict_proba(X_train_scaled)
test_imputed[MY_NRML_PREDICT] = my_lr_normlalized.predict(X_test_scaled, threshold = THRESHOLD)
test_imputed[MY_NRML_PREDICT_PROBA] = my_lr_normlalized.predict_proba(X_test_scaled)

train_imputed[SKLEARN_PREDICT] = sk_lr.predict(X_train)
train_imputed[SKLEARN_PREDICT_PROBA] = sk_lr.predict_proba(X_train)[:, 1]
test_imputed[SKLEARN_PREDICT] = sk_lr.predict(X_test)
test_imputed[SKLEARN_PREDICT_PROBA] = sk_lr.predict_proba(X_test)[:, 1]

### Обработка пропущенных данных

In [ ]:
numeric_features = ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 
                    'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']

numeric_imputer = SimpleImputer(strategy='median')

train_imputed[numeric_features] = numeric_imputer.fit_transform(train[numeric_features])
test_imputed[numeric_features] = numeric_imputer.transform(test[numeric_features])

### Сравнение метрик

In [ ]:
print("MyLogisticRegression weights:", my_lr.weights)
print("Sklearn LogisticRegression weights:", np.append(sk_lr.intercept_, sk_lr.coef_[0]))

In [ ]:
metrics = {}
models = ['my_nrml', 'sk']

for model in models:
    metrics[model] = {
        'accuracy': accuracy_score(test_imputed[TARGET], test_imputed[f'{model}_predict']),
        'precision': precision_score(test_imputed[TARGET], test_imputed[f'{model}_predict']),
        'recall': recall_score(test_imputed[TARGET], test_imputed[f'{model}_predict']),
        'roc_auc': roc_auc_score(test_imputed[TARGET], test_imputed[f'{model}_predict_proba'])
    }

metrics_df = pd.DataFrame(metrics).T
print("\nMetrics comparison:")
print(metrics_df)

### Сравнение матриц ошибок для моделей

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
ConfusionMatrixDisplay(confusion_matrix(test_imputed[TARGET], test_imputed[MY_NRML_PREDICT])).plot(ax=axes[0])
axes[0].set_title('MyLogisticRegression(normalized data)')
ConfusionMatrixDisplay(confusion_matrix(test_imputed[TARGET], test_imputed[SKLEARN_PREDICT])).plot(ax=axes[1])
axes[1].set_title('Sklearn LogisticRegression')
plt.tight_layout()
plt.show()

### Распределения вероятностей

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(test_imputed[MY_NRML_PREDICT_PROBA], bins=50, alpha=0.5, label='My LogReg (with standardized data)')
plt.hist(test_imputed[SKLEARN_PREDICT_PROBA], bins=50, alpha=0.5, label='Sklearn LogReg')
plt.title('Probability Distributions')
plt.xlabel('Predicted Probability')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Сравнение моделей

In [ ]:
# иногда визуальные деффекты 
test_sample = test_imputed.sample(2500)
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
cmap = sns.color_palette('ch:', as_cmap=True)
import matplotlib.colors as mcolors
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mcolors.Normalize())
scatter = sns.scatterplot(data=test_sample, x=feature_1, y=feature_2, hue=TARGET, 
                         hue_norm=sm.norm, palette=cmap)
plt.colorbar(sm, ax=plt.gca()).set_label('Target')
plt.title('Actual Target Values')

plt.subplot(2, 2, 2)
scatter = sns.scatterplot(data=test_sample, x=feature_1, y=feature_2, hue=MY_NRML_PREDICT_PROBA, 
                         hue_norm=sm.norm, palette=cmap)
plt.colorbar(sm, ax=plt.gca()).set_label('My LogReg Probability')
plt.title('MyLogisticRegression Probabilities')

plt.subplot(2, 2, 3)
scatter = sns.scatterplot(data=test_sample, x=feature_1, y=feature_2, hue=SKLEARN_PREDICT_PROBA, 
                         hue_norm=sm.norm, palette=cmap)
plt.colorbar(sm, ax=plt.gca()).set_label('Sklearn LogReg Probability')
plt.title('Sklearn LogisticRegression Probabilities')

plt.subplot(2, 2, 4)
test_sample['my_predict_threshold'] = (test_sample[MY_NRML_PREDICT_PROBA] > 0.5).astype(int)
test_sample['comparison'] = np.where(
    (test_sample['my_predict_threshold'] == test_sample[SKLEARN_PREDICT]) & 
    (test_sample['my_predict_threshold'] == test_sample[TARGET]), 
    'Both Correct', 
    np.where(
        (test_sample['my_predict_threshold'] != test_sample[SKLEARN_PREDICT]) & 
        (test_sample['my_predict_threshold'] == test_sample[TARGET]), 
        'MyLogRegr Correct', 
        np.where(
            (test_sample['my_predict_threshold'] != test_sample[SKLEARN_PREDICT]) & 
            (test_sample[SKLEARN_PREDICT] == test_sample[TARGET]), 
            'Sklearn Correct', 
            'Both Wrong'
        )
    )
)
scatter = sns.scatterplot(data=test_sample, x=feature_1, y=feature_2, hue='comparison', 
                         palette={'Both Correct': 'green', 'MyLogRegr Correct': 'blue', 
                                  'Sklearn Correct': 'orange', 'Both Wrong': 'red'})
plt.title('Comparison of Predictions')
plt.legend(title='Prediction Comparison')

plt.tight_layout()
plt.show()

## Различные пороги классификации

In [ ]:
thresholds = np.linspace(0.01, 0.99, 99)
stats = []

for threshold in thresholds:
    my_pred = (test_imputed[MY_NRML_PREDICT_PROBA] > threshold).astype(int)
    sk_pred = (test_imputed[SKLEARN_PREDICT_PROBA] > threshold).astype(int)
    
    stats.append({
        'threshold': threshold,
        'my_accuracy': accuracy_score(test_imputed[TARGET], my_pred),
        'sk_accuracy': accuracy_score(test_imputed[TARGET], sk_pred),
        'my_precision': precision_score(test_imputed[TARGET], my_pred, zero_division=0),
        'sk_precision': precision_score(test_imputed[TARGET], sk_pred, zero_division=0),
        'my_recall': recall_score(test_imputed[TARGET], my_pred, zero_division=0),
        'sk_recall': recall_score(test_imputed[TARGET], sk_pred, zero_division=0),
    })

In [ ]:
stats_df = pd.DataFrame(stats)

plt.figure(figsize=(10, 5))
sns.lineplot(data=stats_df, x='threshold', y='my_accuracy', label='My LogReg Accuracy')
sns.lineplot(data=stats_df, x='threshold', y='sk_accuracy', label='Sklearn LogReg Accuracy')
plt.title('Accuracy vs Threshold')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data=stats_df, x='threshold', y='my_precision', label='My LogReg Precision')
sns.lineplot(data=stats_df, x='threshold', y='sk_precision', label='Sklearn LogReg Precision')
plt.title('Precision vs Threshold')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data=stats_df, x='threshold', y='my_recall', label='My LogReg Recall')
sns.lineplot(data=stats_df, x='threshold', y='sk_recall', label='Sklearn LogReg Recall')
plt.title('Recall vs Threshold')
plt.grid(True)
plt.legend()
plt.show()

## Сравнение показателей моделей в зависимости от нормировки данных

In [ ]:
NEW_LR = 0.00001
NEW_C_EPOCH = 100

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3-ая версия
my_lr_scaled = MyLogisticRegression(NEW_LR)
my_lr_scaled.fit(X_train_scaled, y_train, NEW_C_EPOCH)

sk_lr_scaled = LogisticRegression(max_iter=1000)
sk_lr_scaled.fit(X_train_scaled, y_train)

train_imputed[MY_SCALED_PREDICT_PROBA] = my_lr_scaled.predict_proba(X_train_scaled)
test_imputed[MY_SCALED_PREDICT_PROBA] = my_lr_scaled.predict_proba(X_test_scaled)
train_imputed[SK_SCALED_PERDICT_PROBA] = sk_lr_scaled.predict_proba(X_train_scaled)[:, 1]
test_imputed[SK_SCALED_PERDICT_PROBA] = sk_lr_scaled.predict_proba(X_test_scaled)[:, 1]

In [ ]:
print("\nScaled MyLogisticRegression weights:", my_lr_scaled.weights)
print("Scaled Sklearn LogisticRegression weights:", np.append(sk_lr_scaled.intercept_, sk_lr_scaled.coef_[0]))

print("\nROC AUC on original data:")
print("My LogReg:", roc_auc_score(test_imputed[TARGET], test_imputed[MY_PREDICT_PROBA]))
print("Sklearn LogReg:", roc_auc_score(test_imputed[TARGET], test_imputed[SKLEARN_PREDICT_PROBA]))

print("\nROC AUC on scaled data:")
print(f"My LogReg with number of epoch: {C_EPOCH} and with learning rate: {LR} :", roc_auc_score(test_imputed[TARGET], test_imputed[MY_NRML_PREDICT_PROBA]))
print(f"My LogReg with number of epoch: {NEW_C_EPOCH} and with learning rate: {NEW_LR} :", roc_auc_score(train_imputed[TARGET], train_imputed[MY_SCALED_PREDICT_PROBA]))
print("Sklearn LogReg:", roc_auc_score(train_imputed[TARGET], train_imputed[SK_SCALED_PERDICT_PROBA]))

## Клиентские/Продуктовые/Кредитные факторы 

In [ ]:
if 'person_emp_length' not in train_imputed.columns or 'person_emp_length' not in test_imputed.columns:
    train_imputed['person_emp_length'] = train['person_emp_exp'] if 'person_emp_exp' in train.columns else np.nan
    test_imputed['person_emp_length'] = test['person_emp_exp'] if 'person_emp_exp' in test.columns else np.nan

client_features = ['person_age', 'person_income', 'person_emp_length']
product_features = ['loan_percent_income', 'loan_int_rate', 'loan_percent_income']
credit_history_features = ['cb_person_cred_hist_length']

group_models = {}
group_predictions = {}

for group_name, group_features in [
    ('client', client_features),
    ('product', product_features),
    ('credit_history', credit_history_features)
]:
    X_train_group = train_imputed[group_features].values
    X_test_group = test_imputed[group_features].values
    
    scaler = StandardScaler()
    X_train_group_scaled = scaler.fit_transform(X_train_group)
    X_test_group_scaled = scaler.transform(X_test_group)
    
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train_group_scaled, y_train)
    
    group_models[group_name] = model
    
    train_imputed[f'{group_name}_predict_proba'] = model.predict_proba(X_train_group_scaled)[:, 1]
    test_imputed[f'{group_name}_predict_proba'] = model.predict_proba(X_test_group_scaled)[:, 1]
    
    print(f"\nROC AUC for {group_name} features:")
    print(roc_auc_score(test_imputed[TARGET], test_imputed[f'{group_name}_predict_proba']))

In [ ]:
meta_features_train = train_imputed[['client_predict_proba', 'product_predict_proba', 'credit_history_predict_proba']].values
meta_features_test = test_imputed[['client_predict_proba', 'product_predict_proba', 'credit_history_predict_proba']].values

meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(meta_features_train, y_train)

train_imputed['meta_predict_proba'] = meta_model.predict_proba(meta_features_train)[:, 1]
test_imputed['meta_predict_proba'] = meta_model.predict_proba(meta_features_test)[:, 1]

In [ ]:
print("\nROC AUC for meta model:")
print(roc_auc_score(test_imputed[TARGET], test_imputed['meta_predict_proba']))

print("\nFinal ROC AUC comparison:")
print("Client features:", roc_auc_score(test_imputed[TARGET], test_imputed['client_predict_proba']))
print("Product features:", roc_auc_score(test_imputed[TARGET], test_imputed['product_predict_proba']))
print("Credit history features:", roc_auc_score(test_imputed[TARGET], test_imputed['credit_history_predict_proba']))
print("Meta model:", roc_auc_score(test_imputed[TARGET], test_imputed['meta_predict_proba']))
print("Full feature space (sklearn):", roc_auc_score(test_imputed[TARGET], test_imputed[SKLEARN_PREDICT_PROBA]))

## График качества (ROC AUC) модели на фиксированной тренировочной выборке в зависимости от количества наблюдений

In [ ]:
sample_sizes = [10, 75, 100, 500, 1000, 2000, 5000, 7500, 10000]
roc_auc_scores = []

for size in sample_sizes:
    train_sample = train_imputed.sample(min(size, len(train_imputed)), random_state=42)
    X_train_sample = train_sample[features].values
    y_train_sample = train_sample[TARGET].values
    
    lr_sample = LogisticRegression(max_iter=1000)
    lr_sample.fit(X_train_sample, y_train_sample)

    test_imputed[f'sample_{size}_predict_proba'] = lr_sample.predict_proba(X_test)[:, 1]
    
    roc_auc = roc_auc_score(test_imputed[TARGET], test_imputed[f'sample_{size}_predict_proba'])
    roc_auc_scores.append({'sample_size': size, 'roc_auc': roc_auc})

roc_auc_df = pd.DataFrame(roc_auc_scores)
plt.figure(figsize=(10, 6))
sns.lineplot(data=roc_auc_df, x='sample_size', y='roc_auc', marker='o')
plt.title('ROC AUC vs Training Sample Size')
plt.xlabel('Training Sample Size')
plt.ylabel('ROC AUC on Test Set')
plt.grid(True)
plt.tight_layout()
plt.show()